In [1]:
%load_ext autoreload
%autoreload 2
# So code is automatically reloaded when saved in different modules

In [2]:
  
import os
import pickle
import numpy as np
import tensorflow as tf
from scipy import stats
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import time
import logging
import sys
import time
from datetime import datetime
import csv
from collections import Counter, OrderedDict, defaultdict
import sys
sys.path.append('src/taggerSystem/')
from my_util import print_sentence, write_conll, read_conll
from my_data_util import load_and_preprocess_data, load_embeddings, ModelHelper
logger = logging.getLogger("hw3.q2")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

## File Paths


In [3]:
data_train = "data/icd9NotesDataTable_train.csv"
data_valid = "data/icd9NotesDataTable_valid.csv"
# data_train = "data/smallIcd9NotesDataTable_test.csv"
# data_valid = "data/smallIcd9NotesDataTable_valid.csv"
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
output_path = 'results/{}/{:%Y%m%d_%H%M%S}/".format(self.cell, datetime.now())'
log_output = output_path + "log"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50

## Extract Data and Word Embeddings

In [ ]:
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid)
# print('dev raw')
# print(dev_raw)
# print('dev')
# print(dev)
embeddings = load_embeddings(vocab, wordVecs, helper)
helper.save(output_path)# token2id and max length saved to output_path
handler = logging.FileHandler(log_output)
handler.setLevel(logging.DEBUG)
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
logging.getLogger().addHandler(handler)

INFO:Loading training data...
INFO:Done. Read 39544 notes
INFO:Loading dev data...
INFO:Done. Read 13182 notes
INFO:Total read time 24.944909


## Hyperparameters

In [5]:
tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 100
batch_size = 1
total_batches = (xTrain.shape[0]//batch_size)

n_input = 1
n_steps = 10
n_hidden = 30
n_classes = helper.n_labels
n_features = 1

alpha = 0.5

In [6]:
embeddings[0].shape

(50,)

In [7]:
helper.max_length

5

## Placeholders

In [8]:
# x = tf.placeholder(tf.int32, shape= (None, helper.max_length, n_features))
x = tf.placeholder(tf.int32, shape= (None, helper.max_length))
yTruth = tf.placeholder(tf.int32, shape = (None, helper.n_labels))
y_steps = tf.placeholder(tf.int32, shape = (None, helper.n_labels))# not sure what this is

## Helper Functions

In [9]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden,state_is_tuple = True)
    print(type(cell))
    print(cell)
    print(type(x))
    print(x.get_shape())
    print('cell output size')
    print(cell.output_size)
    print('cell state size')
    print(cell.state_size)
#     1/0
#     print('cell size')
#     print(cell.get_shape())
#     1/0
#     multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 2)

    output, state = tf.nn.dynamic_rnn(cell = cell, inputs = x, dtype = tf.float32)
    print('output shape')
    print(output.get_shape())
#     print(tf.transpose(output,[1,0,2]).get_shape())
#     temp = tf.gather(tf.transpose(output,[1,0,2]), helper.max_length-1) 
#     print('temp shape')
#     print(temp.get_shape())
#     1/0
    output_flattened = tf.reshape(output, [-1, n_hidden])
    # This isn't getting everything you want. Shrinks from (None, max_len, hiddenSize)
    # to (None, hiddenSize) so not sure what happens to other stuff
    print('output flattened shape')
    print(output_flattened.get_shape())
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
#     output_all = tf.nn.sigmoid(output_logits)
    print('output wx + b')
    print(output_logits.get_shape())
#     1/0
#     output_reshaped = tf.reshape(output_logits,[-1,n_steps,n_classes])
#     print('reshaped shape')
#     print(output_reshaped.get_shape())
    output_reshaped = tf.reshape(output_logits,[-1,helper.max_length,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), helper.max_length - 1)
    #Here is where you wanna get the last one. 
    #output = tf.transpose(output, [1, 0, 2])
    #last = tf.gather(output, int(output.get_shape()[0]) - 1)
    #output_last = tf.nn.sigmoid(tf.matmul(last, weight) + bias)
    return output_last

In [12]:
# weight = weight_variable([n_hidden,n_classes])
with tf.variable_scope('RNN_OutsideCell', reuse = True) as scope:
    U = tf.get_variable(name = 'U', shape = (n_hidden, n_classes), 
                    initializer = tf.contrib.layers.xavier_initializer())
    bias = tf.get_variable(name = 'bias', shape = [n_classes], 
                           initializer = tf.constant_initializer(0))
    # bias = bias_variable([n_classes])
    pretrainedEmbeddings = tf.Variable(embeddings)
    wordEmbeddings = tf.nn.embedding_lookup(params = pretrainedEmbeddings, ids = x)
    print(wordEmbeddings.get_shape())
#     wordEmbeddings = tf.reshape(wordEmbeddings, 
#                                 shape = tf.stack([-1, wordEmbeddings.get_shape()[1], wordEmbeddings.get_shape()[2]*wordEmbeddings.get_shape()[3]])) 
    print('shape of embeddings')
    print(wordEmbeddings.get_shape())
    print('U shape')
    print(U.get_shape())
    print('bias shape')
    print(bias.get_shape())
    y_last = LSTM(wordEmbeddings,U,bias)
# adding loss function

(?, 5, 50)
shape of embeddings
(?, 5, 50)
U shape
(30, 4)
bias shape
(4,)
<class 'tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.BasicLSTMCell'>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 5, 50)
cell output size
30
cell state size
LSTMStateTuple(c=30, h=30)
output shape
(?, 5, 30)
output flattened shape
(?, 30)
output wx + b
(?, 4)


## Loss Function

In [13]:
batchError = tf.nn.sigmoid_cross_entropy_with_logits(logits = y_last, 
                                                 labels = tf.cast(yTruth, tf.float32))
# print('yolo2')
loss_function = tf.reduce_mean(batchError)
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)

/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
# The paper does something weird. I'd suggest doing something closer to what we do in homework 3

## Training and Testing

In [18]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
# hacky work around for issue where -1 is padding but now maps to UUNNNKKK vector.

In [25]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (yTrain.shape[0] - batch_size)
            batch_x = xTrain[offset:(offset + batch_size), :]
#             print('batch x size')
#             print(batch_x.shape)
            batch_y = yTrain[offset:(offset + batch_size), :]
#             print('y shape')
#             print(batch_y.shape)
            batch_y_steps = np.tile(batch_y,((xTrain.shape[1]),1))
#             1/0
            _, c = session.run([train_op, loss_function],feed_dict={x: batch_x, yTruth : batch_y})
#             1/0
            print(c)
        pred_y = session.run(y_last,feed_dict={x:xDev})
        print("ROC AUC Score: ",roc_auc_score(yDev,pred_y))

0.718697
0.695526
0.667659
0.686773
0.697436
0.688438
0.668014
ROC AUC Score:  0.65
0.660136
0.702406
0.609767
0.64017
0.676622
0.66857
0.618395
ROC AUC Score:  0.708333333333
0.610994
0.715854
0.558348
0.595612
0.655317
0.647694
0.571085
ROC AUC Score:  0.791666666667
0.563943
0.729229
0.510523
0.551836
0.632792
0.625446
0.525006
ROC AUC Score:  0.791666666667
0.517939
0.74157
0.46533
0.507936
0.60821
0.601183
0.479282
ROC AUC Score:  0.791666666667
0.472181
0.751918
0.42239
0.463219
0.580662
0.574156
0.433542
ROC AUC Score:  0.833333333333
0.426394
0.758726
0.381836
0.417348
0.549233
0.543589
0.388058
ROC AUC Score:  0.875
0.380957
0.75965
0.344064
0.37047
0.513114
0.508759
0.343713
ROC AUC Score:  0.875
0.336839
0.751637
0.309384
0.323289
0.471771
0.469134
0.301704
ROC AUC Score:  0.875
0.295256
0.731565
0.277717
0.277001
0.425222
0.424613
0.26305
ROC AUC Score:  0.875
0.257175
0.697577
0.248613
0.233095
0.374375
0.375834
0.228251
ROC AUC Score:  1.0
0.223011
0.650525
0.221656
0.193